<a href="https://colab.research.google.com/github/Rohan-Mahajan/LLMs/blob/main/FineTunedLlama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers
!pip install -U datasets
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    HfArgumentParser,
)
from peft import(
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
base_model = "meta-llama/Llama-3.2-1B"
new_model = "llama-3-testing-expert"

In [5]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [6]:
#QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch_dtype,
)

#Load Model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    device_map = "auto",
    attn_implementation = attn_implementation,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [7]:
#Load Tokanizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code = True)
model, tokenizer = setup_chat_format(model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [8]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [9]:
import pandas as pd
from datasets import Dataset

df = pd.read_parquet("hf://datasets/kumarharsh1812/testcases/testcases.parquet")

df_sel = df[['topic', 'test_scenario', 'test_steps', 'expected_intended_result', 'pass_fail_criteria']]

df_cleaned = df_sel.dropna()

dataset = Dataset.from_pandas(df_cleaned)
dataset = dataset.shuffle(seed=0)

In [11]:
def format_chat_template(row):
    combined_text = (
        f"Test Scenario: {row['test_scenario']}\n"
        f"Test Steps:\n{row['test_steps']}\n"
        f"Expected Result: {row['expected_intended_result']}\n"
        f"Pass/Fail Criteria: {row['pass_fail_criteria']}"
    )
    row["Sample"] = combined_text

    row_json = [{"role": "user", "content": row["topic"]},
               {"role": "assistant", "content": row["Sample"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)
dataset['text'][0]

Map (num_proc=4):   0%|          | 0/4673 [00:00<?, ? examples/s]

'<|im_start|>user\nGenerating test cases for social networking app account management to verify that users can view suggested events based on group interests.<|im_end|>\n<|im_start|>assistant\nTest Scenario: Users cannot view suggested events if they have no mutual group members.\nTest Steps:\n1. Login to the social networking app using valid credentials.\n2. Navigate to the events section.\n3. Verify that no suggested events are displayed if there are no mutual group members.\nExpected Result: No suggested events are displayed if there are no mutual group members.\nPass/Fail Criteria: No suggested events are displayed<|im_end|>\n'

In [12]:
dataset = dataset.train_test_split(test_size=0.1)

In [19]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip uninstall transformers
!pip install transformers==4.45.2

Found existing installation: transformers 4.48.0
Uninstalling transformers-4.48.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.11/dist-packages/transformers-4.48.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.48.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.13.0 requires transformers>=4.46.0, but you have transformers 4.45.2 which is in

In [20]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    # max_seq_length=512,
    # dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    # packing= False,
)

<ipython-input-20-a94ccfc6dc06>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/4205 [00:00<?, ? examples/s]

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

In [21]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
421,2.334800,1.056729
842,2.546600,0.958209
1263,1.818900,0.871337
1684,1.277600,0.818772


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: Use

TrainOutput(global_step=2102, training_loss=1.909405942704993, metrics={'train_runtime': 1164.6641, 'train_samples_per_second': 3.61, 'train_steps_per_second': 1.805, 'total_flos': 2942664539885568.0, 'train_loss': 1.909405942704993, 'epoch': 0.9997621878715814})

In [23]:
messages = [
    {
        "role": "user",
        "content": "Generate test cases for social networking app profile management: verify that users can update their cover photo."
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False,
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True,
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150,
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


Test Scenario: Verify that a user can successfully update their cover photo.
Test Steps:
1. Log in to the social networking app.
2. Navigate to the profile section.
3. Click on the option to edit the cover photo.
4. Upload a new cover photo.
5. Save the changes.
Expected Result: The cover photo is updated successfully.
Pass/Fail Criteria: The cover photo is displayed correctly on the user's profile. Pass if the cover photo is updated, fail if the cover photo is not updated. Pass if the cover photo is displayed correctly, fail if the cover photo is not displayed


In [24]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Rohan0707/llama-3-testing-expert/commit/b69c2d71a579dad2c27a013bf8e020cf7c38e631', commit_message='Upload model', commit_description='', oid='b69c2d71a579dad2c27a013bf8e020cf7c38e631', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rohan0707/llama-3-testing-expert', endpoint='https://huggingface.co', repo_type='model', repo_id='Rohan0707/llama-3-testing-expert'), pr_revision=None, pr_num=None)